<a href="https://colab.research.google.com/github/botmaker00/Rex-Deploy/blob/main/rex_hk_deploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
  <img src="https://files.catbox.moe/4w80av.jpg" height="250" alt="Rex Bots Logo"/>
  <h1>🔥 REX ENCODER BOT - HEROKU DEPLOYER 🔥</h1>
  <h3>Ab sirf 5 minute mein apna powerful encoder bot live karo!</h3>
  <b>Channel:</b> @RexBots_Official | <b>Support:</b> @rexbotschat | <b>Developer:</b> @botmaker00
  <br>
  <h2>✨ 3 Tarike se Config Add Karo:</h2>
  <h3>1. Manual Fill • 2. Gist URL • 3. Upload config.env</h3>
</div>

In [ ]:
#@title <font color="#FF0000">🔐 Step 1: Heroku Login (Sabse Pehle Ye Karo!)</font>
Heroku_Email = ""  #@param {type:"string"}
Heroku_API_Key = ""  #@param {type:"string"}
from IPython.display import clear_output, HTML
import os
!curl -s https://cli-assets.heroku.com/install.sh | sh
clear_output()
if not Heroku_Email or not Heroku_API_Key:
    display(HTML('<h3><font color="red">❌ Heroku Email aur API Key daalo!</font></h3>'))
else:
    netrc_content = f"machine api.heroku.com\n  login {Heroku_Email}\n  password {Heroku_API_Key}\nmachine git.heroku.com\n  login {Heroku_Email}\n  password {Heroku_API_Key}"
    with open(os.path.expanduser("~/.netrc"), "w") as f:
        f.write(netrc_content)
    os.chmod(os.path.expanduser("~/.netrc"), 0o600)
    !git config --global user.email {Heroku_Email}
    !git config --global user.name "RexBot_User"
    display(HTML('<h2><font color="lime">✅ Heroku Login Successful!</font></h2>'))

In [ ]:
#@title <font color="#00FFFF">🚀 Step 2: Naya Heroku App Banaye?</font>
App_Names = ""  #@param {type:"string"}
Region = "eu"  #@param ["eu", "us"]
Team_Name = ""  #@param {type:"string"}
if App_Names.strip():
    team_flag = f"--team {Team_Name}" if Team_Name else ""
    for name in App_Names.split():
        name = name.strip().lower()
        if name:
            !heroku create {name} --region {Region} --stack container {team_flag}
            print(f"✅ App bana diya: https://{name}.herokuapp.com")
else:
    print("ℹ️ Koi app name nahi dala, purana app use kar sakte ho.")

In [ ]:
#@title <font color="#FF6600">⚙️ Step 3: Config Setup</font>
App_Name = "rexencoder"  #@param {type:"string"}
API_ID = 123456
API_HASH = ""
BOT_TOKEN = ""
OWNER_ID = 123
SUDO_USERS = ""
LOG_CHANNEL = -100
SESSION_NAME = "rexencoder"
MONGO_URI = ""
Gist_URL = ""
Upload_Config_File = False
import os, shutil
from IPython.display import clear_output, HTML
if os.path.exists(App_Name): shutil.rmtree(App_Name)
!git clone https://github.com/botmaker00/Rex-Deploy {App_Name}
%cd {App_Name}
!rm -f README.md *.ipynb 2>/dev/null || true
if Upload_Config_File:
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys(): os.rename(fn, "config.env")
    clear_output(); display(HTML('<h3><font color=lime>config.env uploaded!</font></h3>'))
elif Gist_URL.strip():
    !curl -L -o config.env "{Gist_URL}"
    clear_output(); display(HTML('<h3><font color=cyan>Gist se config.env download!</font></h3>'))
else:
    with open("config.env", "w") as f:
        f.write(f"API_ID={API_ID}\nAPI_HASH='{API_HASH}'\nBOT_TOKEN='{BOT_TOKEN}'\nOWNER_ID={OWNER_ID}\nSUDO_USERS={SUDO_USERS}\nLOG_CHANNEL={LOG_CHANNEL}\nSESSION_NAME='{SESSION_NAME}'\nMONGO_URI='{MONGO_URI}'\n")
    clear_output(); display(HTML('<h3><font color=gold>Manual config.env ready!</font></h3>'))
%cd ..
display(HTML(f'<h3>{App_Name} ready for deploy!</h3>'))

In [ ]:
#@title <font color="#FF0066">🚀 Step 4: Deploy</font>
Deploy_These_Apps = "rexencoder"
from IPython.display import clear_output, HTML
success = 0
for app in [x.strip() for x in Deploy_These_Apps.split() if x.strip()]:
    if os.path.isdir(app):
        %cd {app}
        !git add . -f
        !git commit -m "Deploy" --allow-empty > /dev/null 2>&1
        !heroku git:remote -a {app} > /dev/null 2>&1 || echo "Remote exists"
        !git push heroku main -f
        print(f"🎉 {app} LIVE ho gaya!")
        success += 1
        %cd ..
    else:
        print(f"❌ Folder nahi mila: {app}")
clear_output()
display(HTML(f'<h1>{success} Bot Deployed!</h1>'))

In [ ]:
#@title 📊 Step 5: Logs
App_Name_For_Logs = ""
if App_Name_For_Logs.strip(): !heroku logs -t -a {App_Name_For_Logs}
else: print("⚠️ App ka naam daalo bhai!")

In [ ]:
#@title 🚪 Step 6: Logout
!heroku logout -i
from IPython.display import HTML
display(HTML('<h2>🔒 Logout Complete</h2>'))

<div align="center"><h2>🎉 Done! Bot Ready!</h2><h3>Made by Already Dead</h3></div>